# Structured output sample


In [25]:
%load_ext autoreload
%autoreload 2

In [26]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from openai import OpenAI
from pydantic import BaseModel, Field

## OpenAI direct


In [2]:
class Step(BaseModel):
    explanation: str = Field(description="Explanation of the step")
    output: str = Field(description="Output of the step")


class MathResponse(BaseModel):
    steps: list[Step] = Field(description="List of steps to solve the math problem")
    final_answer: str = Field(description="Final answer of the math problem")

In [ ]:
client = OpenAI()

In [ ]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor."},
        {"role": "user", "content": "solve 8x + 31 = 2"},
    ],
    response_format=MathResponse,
)

message = completion.choices[0].message
if message.parsed:
    print(message.parsed.steps)
    print(message.parsed.final_answer)
else:
    print(message.refusal)

## Langchain


### Sample


In [3]:
# class Joke(BaseModel):
#     setup: str = Field(description="The setup of the joke")
#     punchline: str = Field(description="The punchline to the joke")

In [8]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = model.with_structured_output(MathResponse)

In [9]:
# structured_llm.invoke("Tell me a joke about cats")
output = structured_llm.invoke(
    [
        {"role": "system", "content": "You are a helpful math tutor."},
        {"role": "user", "content": "solve 8x + 31 = 2"},
    ]
)

In [ ]:
# output: MathResponse = output  # type: ignore
if not isinstance(output, MathResponse):
    raise ValueError("Expected MathResponse but got something else")
for step in output.steps:
    print(step.explanation)
    print(step.output)
print(output.final_answer)

### Converse


In [12]:
from convo_craft.llm.converse import Conversation


model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = model.with_structured_output(Conversation)

In [13]:
hm = HumanMessage(
    """Write a conversation in brazilian portuguese between two persons, \
that should be used to teach the user the language.
The conversation should be about the following topic: "How to order food in a restaurant".
Assume that the user has an intermediate level of understanding of the language.
The conversation should last about 10 messages in total, with each message being about 2-3 sentences long.
"""
)

In [14]:
convo = structured_llm.invoke([hm])

In [ ]:
if not isinstance(convo, Conversation):
    raise ValueError("Expected Conversation but got something else")

for turn in convo.turns:
    print(turn.role)
    print(turn.content)
    print()

### Prompt


In [10]:
# turn the hm into a prompt

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import (
    ChatMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    StringPromptTemplate,
    SystemMessagePromptTemplate,
)

In [ ]:
convo_template = """Write a conversation in {language} between two persons, \
that should be used to teach the user the language.
The conversation should be about the following topic: "{topic}".
Assume that the user has an intermediate level of understanding of the language.
The conversation should last about {num_msg} messages in total, with each message being about 2-3 sentences long.
"""
converse_prompt = ChatPromptTemplate(
    [HumanMessagePromptTemplate.from_template(convo_template)]
)
converse_value = converse_prompt.invoke(
    {
        "language": "brazilian portuguese",
        "topic": "How to order food in a restaurant",
        "num_msg": "10",
    }
)
print(converse_value.to_messages()[0].content)

### Generator


In [30]:
from convo_craft.llm.converse import CONVERSATION_SAMPLE, ConversationGenerator


cg = ConversationGenerator(
    language="brazilian portuguese",
    num_msg=10,
    understanding_level="intermediate",
    conversation_sample=CONVERSATION_SAMPLE,
)
# conv = cg.invoke("A conversation about ordering food in a restaurant.")
conv = cg.invoke("A conversation about traveling to a foreign country.")

In [ ]:
for turn in conv.turns:
    print(turn.content)
    print()

### Translator


In [ ]:
from convo_craft.llm.translate import Translator


tr = Translator(
    source_language="brazilian portuguese",
    target_language="english",
)
translation = tr.invoke("Eu gosto de comer pizza.")
translation

### Splitter


In [ ]:
from convo_craft.llm.split_paragraph import ParagraphSplitter


ps = ParagraphSplitter()
orig = "Eu gosto de comer pizza. Eu também gosto de comer hambúrguer. Assim como gosto de comer batata frita."
split = ps.invoke(orig)
split

In [ ]:
rebuild = " ".join(split.portions)
rebuild == orig

### Topic picker


In [ ]:
from convo_craft.llm.topic_picker import OLD_TOPICS, TopicPicker


tp = TopicPicker(
    language="brazilian portuguese",
    understanding_level="intermediate",
)
topics = tp.invoke(OLD_TOPICS)
for topic in topics.topics:
    print(topic)

### Sentence splitter


In [ ]:
from convo_craft.text.split_sentence import SentenceSplitter


ss = SentenceSplitter()
words = ss.invoke("I like to eat pizza. I also like to eat hamburgers.")
print(" | ".join(words))